In [5]:
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
# import tsdb
# import benchpots
# from pypots.optim import Adam
# from pypots.imputation import CSDI, MRNN, BRITS

# from pypots.utils.random import set_random_seed
# from pypots.utils.metrics import calc_mae
import random
import json

#from cdrec.python.recovery import centroid_recovery as CDrec


In [6]:
import numpy as np

# 1. Import Original Train/Test Split

In [7]:
train = np.loadtxt("ECG200_TRAIN.txt")
test = np.loadtxt("ECG200_TEST.txt");
X_train = train[:, 1:]
y_train = train[:, 0]
X_test = test[:, 1:]
y_test = test[:, 0]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(100, 96)
(100,)
(100, 96)
(100,)


In [8]:
print(len(np.argwhere(y_train == -1).flatten())/len(y_train))

0.31


In [9]:
print(len(np.argwhere(y_test == -1).flatten())/len(y_test))

0.36


# 2. Make 29 resampled folds

In [10]:
Xs = np.vstack([X_train, X_test]) # recombined train and test 
ys = np.concatenate([y_train, y_test])

In [11]:
ss = StratifiedShuffleSplit(n_splits=29, random_state=0, train_size=100)
splits = ss.split(Xs, ys)

In [12]:
fold_idxs = {0: {"train": np.arange(len(y_train)).tolist(), "test": np.arange(len(y_train), len(ys)).tolist()}} # append original dataset as fold 0
for i, (train_idx, test_idx) in enumerate(ss.split(Xs, ys)):
    sub_dict = {"train": train_idx.tolist(), "test": test_idx.tolist()}
    fold_idxs[i+1] = sub_dict   

Check class distributions

In [14]:
for f in fold_idxs.keys():
    train_idxs = fold_idxs[f]["train"]
    test_idxs = fold_idxs[f]["test"]
    class0_tr_counts = len(np.argwhere(ys[train_idxs] == -1))/len(ys[train_idxs])
    class1_tr_counts = len(np.argwhere(ys[train_idxs] == 1))/len(ys[train_idxs])
    class0_te_counts = len(np.argwhere(ys[test_idxs] == -1))/len(ys[test_idxs])
    class1_te_counts = len(np.argwhere(ys[test_idxs] == 1))/len(ys[test_idxs])
    print(f"fold {f} | class 0: {class0_tr_counts} | class 1: {class1_tr_counts}")
    print(f"fold {f} | class 0: {class0_te_counts} | class 1: {class1_te_counts}")
    print("="*50)

fold 0 | class 0: 0.31 | class 1: 0.69
fold 0 | class 0: 0.36 | class 1: 0.64
fold 1 | class 0: 0.33 | class 1: 0.67
fold 1 | class 0: 0.34 | class 1: 0.66
fold 2 | class 0: 0.33 | class 1: 0.67
fold 2 | class 0: 0.34 | class 1: 0.66
fold 3 | class 0: 0.33 | class 1: 0.67
fold 3 | class 0: 0.34 | class 1: 0.66
fold 4 | class 0: 0.34 | class 1: 0.66
fold 4 | class 0: 0.33 | class 1: 0.67
fold 5 | class 0: 0.33 | class 1: 0.67
fold 5 | class 0: 0.34 | class 1: 0.66
fold 6 | class 0: 0.34 | class 1: 0.66
fold 6 | class 0: 0.33 | class 1: 0.67
fold 7 | class 0: 0.34 | class 1: 0.66
fold 7 | class 0: 0.33 | class 1: 0.67
fold 8 | class 0: 0.33 | class 1: 0.67
fold 8 | class 0: 0.34 | class 1: 0.66
fold 9 | class 0: 0.34 | class 1: 0.66
fold 9 | class 0: 0.33 | class 1: 0.67
fold 10 | class 0: 0.34 | class 1: 0.66
fold 10 | class 0: 0.33 | class 1: 0.67
fold 11 | class 0: 0.34 | class 1: 0.66
fold 11 | class 0: 0.33 | class 1: 0.67
fold 12 | class 0: 0.34 | class 1: 0.66
fold 12 | class 0: 0

# 3. Generate window locations

In [15]:
ps = [0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95]

In [24]:
def generate_windows(T, fraction_missing, num_windows, seed=0):
    np.random.seed(0)
    num_pts = round(T*fraction_missing)
    max_start_idx = T - num_pts # inclusive of start_idx
    max_num_windows = len(np.arange(0, max_start_idx))
    num_windows_choose = min(num_windows, max_num_windows)
    start_idx = random.sample(np.arange(0, max_start_idx).tolist(), num_windows_choose)
    windows = [np.arange(sidx, sidx+num_pts).tolist() for sidx in start_idx]
    return windows

In [25]:
window_per_percentage = dict()
for p in ps:
    window_per_percentage[p] = generate_windows(96, p, 15)

In [29]:
window_per_percentage[0.05]

[[57, 58, 59, 60, 61],
 [77, 78, 79, 80, 81],
 [78, 79, 80, 81, 82],
 [27, 28, 29, 30, 31],
 [36, 37, 38, 39, 40],
 [48, 49, 50, 51, 52],
 [51, 52, 53, 54, 55],
 [49, 50, 51, 52, 53],
 [39, 40, 41, 42, 43],
 [74, 75, 76, 77, 78],
 [2, 3, 4, 5, 6],
 [55, 56, 57, 58, 59],
 [50, 51, 52, 53, 54],
 [73, 74, 75, 76, 77],
 [0, 1, 2, 3, 4]]

# 4. Save Data

Save window idxs:

In [30]:
with open('windows_python_idx.json', 'w') as f:
    json.dump(window_per_percentage, f)

Save resample fold idxs:

In [31]:
with open('resample_folds_python_idx.json', 'w') as f:
    json.dump(fold_idxs, f)

# 4. Test